<div style='background-image: url("../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">ObsPy Tutorial</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Handling Waveform Data</div>
        </div>
    </div>
</div>

Seismo-Live: http://seismo-live.org

##### Authors:
* Tobias Megies ([@megies](https://github.com/megies))
* Lion Krischer ([@krischer](https://github.com/krischer))
* Yannik Behr ([@yannikbehr](https://github.com/yannikbehr))
---

![](https://github.com/yannikbehr/BMKG_OBSPY_WORKSHOP/blob/main/images/obspy_logo_full_524x179px.png?raw=1)

In [ ]:
try:
  import obspy
except ModuleNotFoundError:
  !pip -qq install obspy

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

#### Trace Exercises
 - Make an **`numpy.ndarray`** with zeros and (e.g. use **`numpy.zeros()`**) and put an ideal pulse somewhere in it
 - initialize a **`Trace`** object with your data array
 - Fill in some station information (e.g. network, station, ..)
 - Print trace summary and plot the trace
 - Change the sampling rate to 20 Hz
 - Change the starttime of the trace to the start time of this session
 - Print the trace summary and plot the trace again

In [ ]:
import numpy as np
from obspy import Trace, UTCDateTime, read
x = np.zeros(300)
x[100] = 1.0
tr = Trace(data=x)
tr.stats.station = "ABC"
tr.stats.sampling_rate = 20.0
tr.stats.starttime = UTCDateTime(2014, 2, 24, 15, 0, 0)
print(tr)
tr.plot();

- Use **`tr.filter(...)`** and apply a lowpass filter with a corner frequency of 1 Hertz.
- Display the preview plot, there are a few seconds of zeros that we can cut off.

In [ ]:
tr.filter("lowpass", freq=1)
tr.plot();

- Use **`tr.trim(...)`** to remove some of the zeros at start and at the end
- show the preview plot again

In [ ]:
tr.trim(tr.stats.starttime + 3, tr.stats.endtime - 5)
tr.plot();

- Scale up the amplitudes of the trace by a factor of 500
- Add standard normal gaussian noise to the trace (use [**`np.random.randn()`**](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randn.html))
- Display the preview plot again

In [ ]:
tr.data = tr.data * 500
tr.data = tr.data + np.random.randn(len(tr))
tr.plot();

#### Stream Exercises

- Read all example earthquake data into a stream object ("./data/\*.mseed")
- Print the stream summary

In [ ]:
!wget https://github.com/yannikbehr/BMKG_ObsPy_workshop/raw/refs/heads/main/data/indonesia_waveforms.tar.gz
!tar -xzf indonesia_waveforms.tar.gz

In [ ]:
st = read("./data/*.mseed")
print(st)

- Use **`st.select()`** to only keep traces of station BBJI in the stream. Show the preview plot.

In [ ]:
st = st.select(station="BBJI")
st.plot();

- trim the data to a 10 minute time window around the first arrival (just roughly looking at the preview plot)
- display the preview plot and spectrograms for the stream (with logarithmic frequency scale, use `wlen=50` for the spectrogram plot)

In [ ]:
t1 = UTCDateTime(2018, 12, 22, 14, 30)
st.trim(t1, t1 + 10 * 60)
st.plot()
st.spectrogram(log=True, wlen=50);

- remove the linear trend from the data, apply a tapering and a lowpass at 0.1 Hertz
- show the preview plot again

In [ ]:
st.detrend("linear")
st.filter("lowpass", freq=0.1)
st.plot();